In [ ]:
import bs4 as bs
import requests
import urllib.request
import pandas as pd
import re

In [ ]:
url = 'https://www.worldometers.info/coronavirus/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

# Read text result into Pandas
dfs = pd.read_html(r.text)
# Look for the first table
df = dfs[0]
# Fill missing values with 0
df = df.fillna(0)

df.to_csv('./Data/world_info.csv')

In [ ]:
homepage_soup = bs.BeautifulSoup(r.text, 'lxml')
# Using the table on the main page, only the countries with links (<a> tags) have detailed historical data.
country_elements = homepage_soup.select('table[id="main_table_countries_today"] > tbody > tr > td > a')
# Start with an empty list
countries_with_detailed_data = {}
# Iterate through each element, and add the contents (country name) to a list
for individual_element in country_elements:
    country_name = individual_element.contents[0]
    country_url = individual_element['href']
    countries_with_detailed_data[country_name] = country_url

print("Can get detailed data for %s" % ", ".join(list(countries_with_detailed_data.keys())))

In [ ]:
# Looks for script elements that contain the JS stats by day, and extract x and y axis values
def extractDataFromGraph(soup, chart_id):
    scripts = soup.select('script[type="text/javascript"]')
    for individual_script in scripts:
        individual_script = individual_script.get_text(strip=True)
        if(chart_id in individual_script):
            x_text = re.search('categories: \[([^\]]+)\]', individual_script).group(1)
            y_text = re.search('data: \[([0-9, ]+)]', individual_script).group(1)
            x_values = x_text.replace('"', '').split(",")
            y_values = y_text.split(",")
    return (x_values, y_values)

# Iterate over each country and extract the same data, save to CSV
def getDetailedDataForCountry(country, url_part):
    url = 'https://www.worldometers.info/coronavirus/' + url_part
    r = requests.get(url, headers=header).text

    soup = bs.BeautifulSoup(r, 'lxml')
    dates, cases = extractDataFromGraph(soup, "coronavirus-cases-linear")
    # a bit risky ignoring dates for the following, if for some reason they are different on different graphs
    active_cases = extractDataFromGraph(soup, "graph-active-cases-total")[1]
    deaths = extractDataFromGraph(soup, "coronavirus-deaths-linear")[1]
    daily_deaths = extractDataFromGraph(soup, "graph-deaths-daily")[1]
    # doesn't work: graph-cases-daily
    df_country = pd.DataFrame(
    {'Dates': dates,
     'Cases': cases,
     'Deaths': deaths,
     'Active Cases': active_cases,
     'Daily Deaths': daily_deaths
    })
    df_country = df_country.fillna(0)

    file_name = './Data/Countries/' + country + '.csv'
    df_country.to_csv(file_name)

# Execute the functions to collect data for all countries
for country, url in countries_with_detailed_data.items():
    getDetailedDataForCountry(country, url)